In [18]:
import json
import re
import pandas as pd
import glob
import os

# MM 抽出
def extract_major_minor(url):
    m = re.search(r"FMC(\d+)-MC(\d+)", url)
    return (m.group(1), m.group(2)) if m else ("", "")

def extract_gradeid(url):
    m = re.search(r"gradeid/(\d+)", url)
    return m.group(1) if m else ""

# サイズ分解
def parse_size(size_text):
    height = width = length = wheelbase = ""
    def extract_mm(s):
        m = re.search(r"([\d,]+)mm", s)
        return m.group(1).replace(",", "") if m else ""
    lines = size_text.split("\n")
    for i, line in enumerate(lines):
        if "全高" in line:
            height = extract_mm(line)
        elif "全幅" in line:
            width = extract_mm(line)
        elif "全長" in line:
            length = extract_mm(line)
        elif "ホイールベース" in line:
            if i + 1 < len(lines):
                wheelbase = extract_mm(lines[i + 1])
    return {
        "高さ": height,
        "幅": width,
        "長さ": length,
        "ホイールベース": wheelbase
    }


# =============================
#   JSON → CSV のメイン処理
# =============================
folder = r"D:\GitHub_Repository\Research\ma_thesis\data\car_data"
pattern = folder + r"\*.json"   # ← これが超重要（フォルダ内のJSONのみ対象）
rows = []

for filepath in glob.glob(pattern):
    print(f"Processing: {filepath}")

    with open(filepath, "r", encoding="utf-8") as f:
        data = json.load(f)

    maker = data.get("maker", "")
    model = data.get("car", "")

    historical = data["historical_models"]

    for model_url, minor_obj in historical.items():

        # major, minor 抽出
        major, _ = extract_major_minor(model_url)

        minor_changes = minor_obj.get("minor_changes", {})

        for minor_url, grade_dict in minor_changes.items():
            _, minor = extract_major_minor(minor_url)

            for grade_url, specs in grade_dict.items():

                # エンジン区分が EV の場合は除外
                base_info = specs.get("基本情報", {})
                engine_type = base_info.get("エンジン区分", "")

                if engine_type == "EV":
                    continue  # ★ EV をスキップ ★

                gradeid = extract_gradeid(grade_url)

                flat = {
                    "maker": maker,
                    "model": model,
                    "major": major,
                    "minor": minor,
                    "gradeid": gradeid
                }

                # カテゴリ展開
                for category, items in specs.items():

                    if category == "寸法・重量":
                        if "サイズ" in items:
                            size_dict = parse_size(items["サイズ"])
                            flat.update(size_dict)

                        for k, v in items.items():
                            if k != "サイズ" and k.strip() != "":
                                flat[f"寸法・重量_{k}"] = v

                    else:
                        for k, v in items.items():
                            if k.strip() == "":
                                continue
                            flat[f"{category}_{k}"] = v

                rows.append(flat)

# ---------------------------
#   CSV 出力
# ---------------------------
df = pd.DataFrame(rows)
#df.to_csv("all_cars_output2.csv", encoding="utf-8-sig", index=False)

print("Completed! Saved to all_cars_output.csv")


Processing: D:\GitHub_Repository\Research\ma_thesis\data\car_data\abarth_abarth_124_spider.json
Processing: D:\GitHub_Repository\Research\ma_thesis\data\car_data\abarth_abarth_500e_hatchback.json
Processing: D:\GitHub_Repository\Research\ma_thesis\data\car_data\abarth_abarth_500_cabriolet.json
Processing: D:\GitHub_Repository\Research\ma_thesis\data\car_data\abarth_abarth_500_hatchback.json
Processing: D:\GitHub_Repository\Research\ma_thesis\data\car_data\abarth_abarth_595c_cabriolet.json
Processing: D:\GitHub_Repository\Research\ma_thesis\data\car_data\abarth_abarth_595e_cabriolet.json
Processing: D:\GitHub_Repository\Research\ma_thesis\data\car_data\abarth_abarth_595_hatchback.json
Processing: D:\GitHub_Repository\Research\ma_thesis\data\car_data\abarth_abarth_695_cabriolet.json
Processing: D:\GitHub_Repository\Research\ma_thesis\data\car_data\abarth_abarth_695_hatchback.json
Processing: D:\GitHub_Repository\Research\ma_thesis\data\car_data\abarth_abarth_grande_punto.json
Processing:

In [5]:
from collections import defaultdict
import numpy as np
suffix_dict = defaultdict(list)
for col in df.columns:
    if "_" in col:
        suffix = col.split("_", 1)[1]  # 先頭の "_" だけで分割
        suffix_dict[suffix].append(col)

# ---- 2. サフィックスが重複しているカラムだけ抽出 ----
duplicate_suffixes = {sfx: cols for sfx, cols in suffix_dict.items() if len(cols) >= 2}

print("重複しているサフィックス：")
for sfx, cols in duplicate_suffixes.items():
    print(f"  {sfx}: {cols}")

# ---- 3. 各サフィックスについて、値が一致していない行を抽出 ----
diff_results = {}  # サフィックス → 不一致データフレーム

for sfx, cols in duplicate_suffixes.items():
    # カラムが2つ以上あることをチェック
    if len(cols) < 2:
        continue
    
    # まず1列目を基準列にする
    base = cols[0]
    
    # サフィックス内のすべての列について比較
    diff_mask = pd.Series(False, index=df.index)
    
    for col in cols[1:]:
        diff_mask |= (df[base] != df[col])
    
    # 不一致行を保存
    diff_df = df.loc[diff_mask, ["maker", "model"] + cols]
    diff_results[sfx] = diff_df

重複しているサフィックス：
  タイヤサイズ（前）: ['タイヤ・足回り_タイヤサイズ（前）', 'バッテリー・モーター性能_タイヤサイズ（前）']
  タイヤサイズ（後）: ['タイヤ・足回り_タイヤサイズ（後）', 'バッテリー・モーター性能_タイヤサイズ（後）']
  ブレーキシステム（前）: ['タイヤ・足回り_ブレーキシステム（前）', 'バッテリー・モーター性能_ブレーキシステム（前）']
  ブレーキシステム（後）: ['タイヤ・足回り_ブレーキシステム（後）', 'バッテリー・モーター性能_ブレーキシステム（後）']
  サスペンション（前）: ['タイヤ・足回り_サスペンション（前）', 'バッテリー・モーター性能_サスペンション（前）']
  サスペンション（後）: ['タイヤ・足回り_サスペンション（後）', 'バッテリー・モーター性能_サスペンション（後）']
  ABS（アンチロックブレーキ）: ['安全装備_ABS（アンチロックブレーキ）', 'タイヤ・足回り_ABS（アンチロックブレーキ）']
  ESC（横滑り防止装置）: ['安全装備_ESC（横滑り防止装置）', 'タイヤ・足回り_ESC（横滑り防止装置）']
  TCS（トラクションコントロール）: ['安全装備_TCS（トラクションコントロール）', 'タイヤ・足回り_TCS（トラクションコントロール）']
  運転席エアバッグ: ['安全装備_運転席エアバッグ', 'タイヤ・足回り_運転席エアバッグ']
  助手席エアバッグ: ['安全装備_助手席エアバッグ', 'タイヤ・足回り_助手席エアバッグ']
  サイドエアバッグ: ['安全装備_サイドエアバッグ', 'タイヤ・足回り_サイドエアバッグ']
  アイドルストップ: ['安全装備_アイドルストップ', 'タイヤ・足回り_アイドルストップ']
  駐車支援装置: ['安全装備_駐車支援装置', 'タイヤ・足回り_駐車支援装置']
  エアコン: ['快適装備_エアコン', '安全装備_エアコン']
  パワーウィンドウ: ['快適装備_パワーウィンドウ', '安全装備_パワーウィンドウ']
  クルーズコントロール: ['快適装備_クルーズコントロール', '安全装備_クルーズコントロール']
  オーディオ（CD）: 

In [20]:

# ---- 1. 正しいカテゴリ・項目の定義 ----
correct_map = {
    "タイヤ・足回り": [
        "タイヤサイズ（前）", "タイヤサイズ（後）",
        "ブレーキシステム（前）", "ブレーキシステム（後）",
        "サスペンション（前）", "サスペンション（後）",
    ],
    "バッテリー・モーター性能": [
        "バッテリー種類", "バッテリー電力容量", "モーター出力",
        "モータートルク", "EV航続距離", "充電時間目安",
    ],
    "安全装備": [
        "ABS（アンチロックブレーキ）", "ESC（横滑り防止装置）",
        "TCS（トラクションコントロール）", "運転席エアバッグ",
        "助手席エアバッグ", "サイドエアバッグ", "アイドルストップ",
        "駐車支援装置",
    ],
    "快適装備": [
        "エアコン", "パワーウィンドウ", "クルーズコントロール",
        "オーディオ（CD）", "カーナビゲーション",
        "オーディオ（MD）", "テレビチューナー",
        "ステアリング調整", "シートヒーター", "ETC", "スマートキー",
    ],
    "エクステリア": [
        "サンルーフ", "フォグランプ",
        "ディスチャージヘッドライト", "ホイール", "電動ドア",
    ],
    "インテリア": [
        "電動シート", "シート生地",
    ],
}

# ---- 2. 正しい column 名の集合を作成 (カテゴリ_項目名) ----
correct_columns = set()
for category, items in correct_map.items():
    for item in items:
        correct_columns.add(f"{category}_{item}")

# ---- 3. df のすべてのカラムをチェックし、誤分類を検出 ----
column_map = {}  # 誤カラム → 正しいカラム

for col in df.columns:
    if "_" not in col:
        continue
        
    cat, item = col.split("_", 1)
    
    # 正しい場合 → スキップ
    if col in correct_columns:
        continue
    
    # 誤分類の場合（item の候補探し）
    # item がどのカテゴリに属するか探す
    for correct_cat, items in correct_map.items():
        if item in items:
            # 誤っている。正しいカラム名を記録
            correct_col = f"{correct_cat}_{item}"
            column_map[col] = correct_col
            break

# ---- 4. 修正処理：誤カラム → 正しいカラムに統合 ----
for wrong_col, correct_col in column_map.items():
    
    # 正しいカラムが存在しなければ作る
    if correct_col not in df.columns:
        df[correct_col] = np.nan
    
    # NaNでない方の値を採用
    df[correct_col] = df[correct_col].combine_first(df[wrong_col])

# ---- 5. 修正後、誤カラムは削除 ----
df_fix = df.drop(columns=list(column_map.keys()))

# ---- 6. 完成した df を保存 ----
df_fix.to_csv("all_cars_output2.csv", index=False)